In [14]:
import yfinance as yf
from data.fetch_data import fetch_pdf_info, fetch_data_from_db
import pandas as pd

sql = """
    select etf_tkr from os_index_info;
"""

os_index_info = fetch_data_from_db(query=sql)

In [15]:
etf_tkr_list = pd.DataFrame(os_index_info).etf_tkr.to_list()

In [16]:
from tqdm.auto import tqdm
import json


my_etf_tkr = "BKCH"

def get_tkr_list(my_etf_tkr):

    pdf_info = fetch_pdf_info(etf_tkr=my_etf_tkr)

    pdf_df = pd.DataFrame(pdf_info)

    tkr_list = pdf_df.sort_values(by="float_shares", ascending=False).head().reset_index(drop=True)['child_stk_tkr'].to_list()

    return tkr_list

for etf_tkr in tqdm(etf_tkr_list):

    tkr_list = get_tkr_list(my_etf_tkr=etf_tkr)

    stk_infos = yf.Tickers(tkr_list)

    dict_list = []

    for tkr in tqdm(tkr_list):

        a_tkr_dict = {}

        a_tkr_dict['stock'] = tkr

        a_tkr_dict['news_titles'] = []

        for news in stk_infos.tickers[tkr].news:

            a_tkr_dict['news_titles'].append(news['title'])

        dict_list.append(a_tkr_dict)

    etf_news_dict = {}
    etf_news_dict[f'{etf_tkr}_news'] = dict_list

    with open(f'./etf_news_infos/{etf_tkr}.json', 'w') as outfile:
        json.dump(etf_news_dict, outfile, indent='\t', ensure_ascii=False)

100%|██████████| 34/34 [01:11<00:00,  2.11s/it]
